## 0. Initialisering
For at eksemplene i notebooken skal fungere smertefritt, må openai og dotenv (python-dotenv)-bibliotekene være installert i miljøet der du starter notebooken. Om nødvendig kan du kjøre `!pip install <library>` i en kode-celle for å installere et manglende bibliotek. 

Før vi begynner å gjøre noe interessant, må vi importere noen bibliotek og metoder, og vi må lese inn en gyldig openai-nøkkel (leses her fra en lokal .env - fil).

In [ ]:
# bibliotek vi trenger i dette kapittelet
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import numpy as np
import tiktoken
import json
from pprint import pprint
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from IPython.display import display, HTML

# forenklende pandas-setting
pd.options.mode.copy_on_write = True

In [ ]:
# leser API-nøkkel og initialiserer OpenAI-klient

_ = load_dotenv(find_dotenv()) # leser fra i lokal .env-fil
openai.api_key  = os.environ['OPENAI_API_KEY']

client=OpenAI()

## 1. Grunnleggende bruk av ChatCompletions-endepunktet

### 1.1 Kalle API-et og få en respons...  
For å få API-et til å generere en respons på et spørsmål i naturlig språk, må vi...
 - initialisere klienten
 - spesifisere hvilken språkmodell vi ønsker å bruke, med "model" - parameteren (se [OpenAI model docs](https://platform.openai.com/docs/models/overview) for oppdaterte detaljer)
 - initialisere "samtalen" med "message" - parameteren

In [ ]:
# Gjør en første forespørsel til API

# forespørsel til modellen gis gjennom "messages" -parameteren
messages = [  
{"role": "system", "content": "You are a helpful tourist information agent in Oslo, Norway."},    
{"role": "user", "content": "Hvor finner jeg det berømte vikingskipsmuseet?"}  
] 

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

pprint(response.choices[0].message.content, width=80)

### 1.2 Noen nyttige chat.completions.create - parametre: *max_tokens, temperature, top_p* og *n*   
***temperature***: justerer "kreativiteten" i responsen: høyere temperatur gir mer fantasifulle og varierte svar, men også høyere forekomst av feil og hallusinasjoner. Default=1, max=2, min=0. For stabile og (ganske) strengt faktabaserte svar, velg *temperature*=0. Høye temperature-verdier kan innimellom gi lang responstid.  
***top_p***: cutoff sannsinlighetsverdi for predikerte tokens. En lavere verdi innebærer et strengere "filter" for å legge til nye tokens i responsen og gir dermed kortere, mindre varierte og mer konsise svar. Default=1. 
Merk: openAI anbefaler å bare bruke en av parametrene *temperature* og *top_p* for å tune svarene, og la den andre beholde defaultverdien (=1).  
***max_tokens***: grense for antall tokens brukt (i prompt og respons samlet).  
***n***: antall responser returnert. I noen situasjoner kan det være hensikstsmessig å generere flere svar, og så velge det "beste" ut fra et eller flere vurderingskriterier. 

In [ ]:
# bruk av temperature eller top_p for å justere "kreativitet"
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    top_p=1
)
pprint(response.choices[0].message.content, width=80)

In [ ]:
# flere responser for samme forespørsel - illustrerer effekten av temperature
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    n=3
)
for c in response.choices:
    pprint('Svar nr ' + str(c.index+1) + ": "+  c.message.content, width=80)
    print('\n')

In [ ]:
# temperature=0 er ikke noen garanti mot hallusinasjoner
# Justering prompt: You are very factual. If you do not know something say you do not know.

messages = [  
{"role": "system", "content": "You are a tourist information agent in Oslo, Norway. "},    
{"role": "user", "content": "Hvor finner jeg hovedkontoret til Sparebank 1 Utvikling?"}  
] 

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
)

pprint(response.choices[0].message.content, width=80)

### 1.3 Flere potensielt nyttige chat.completions.create - parametre 
***frequency_penalty***: Reduserer sannsynligheten for å legge til tokens i forhold til hvor ofte de forekommer i den foregående teksen. Skala: -2 til 2, høyere positiv verdi indikerer strengere "straff" for høy forekomst. <br/>
***presence_penalty***: Reduserer sannsynligheten for å legge til tokens som allerede forekommer i den foregående teksten. Skala: -2 til 2, høyere positiv verdi indikerer strengere straff for tidligere forekomst. <br/>
***stop***: kan brukes til å la utvalgte ord eller ordkombinasjoner avslutte svaret, hvis de forekommer.<br/>
***seed***: (beta-funksjonalitet) lar deg sette seed-verdi for å generere et deterministisk utvalg. NB - denne funksjonaliteten er i beta, og det er ingen garanti for at samplingen faktisk blir fullstendig deterministisk.   


## 2. Chat completions - objektet (responsen du får når du kaller chat.completions.create) 

Chat completions - objektet har mange elementer, mange av dem inneholder imidlertid bare relativt uinteressant referanseinformasjon. Se eventuelt [OpenAI API-dokumentasjon](https://platform.openai.com/docs/api-reference/chat/object) for en generell og oppdatert oversikt.

Her vil vi fokusere på listen *choices* og raskt nevne listen *usage* .

<img src="resources/chat_completion.png" alt="OpenAI chatCompletion" />

**usage** (liste) inneholder en opptelling av antall tokens brukt i prompt, respons og til sammen. Dette kan være nyttig informasjon for kostnadskontroll, og for å forstå situasjoner der begrensningene på antall tokens i kontekst gir problemer. 

In [ ]:
# Sjekk av "usage" 
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    top_p=1
)
pprint(dict(response.usage))


***choices*** - listen inneholder responsen(e) fra modellen.<br/> 
Hvert ***choices[index]***- listeelement inneholder igjen disse elementene: 
 - finish_reason: indikerer hvorfor svaret ble avsluttet, og kan være nyttig å undersøke i feilsituasjoner
 - logprobs: gir sannsynlighetsverdier for hver token i det genererte svaret
 - message (liste): den genererte responsen, med tilleggsinformasjon om hvordan den har blitt generert


In [ ]:
# innholdet i et element fra "choices" 
pprint(dict(response.choices[0]), depth=1, width=80)

***choices[index].message*** - listen inneholder responsteksten i tillegg til informasjon om hvordan teksten ble generert: 
- role: "rollen" som gir svaret (som oftest "assistant") 
- content: selve den generert teksten 
- tool_calls: beskrivelse av eventuelle "tool calls" (se under) som har blitt brukt for å lage svaret  

Merk at innholdet av "messages" fra kallet til chat.completions.create IKKE er inkludert i responsen, kun (nyeste) output fra modellen. Hvis du ønsker å beholde oversikt over en flerstegs dialog må du i utgangspunktet selv bygge opp og ta vare på historikken.


In [ ]:
# innholdet i "messages" for et "choices" - element
pprint(dict(response.choices[0].message), depth=1, width=80)

## 3. Tools og Tool Calls - ekstra superkrefter til språkmodellen 
ChatCompletions-APIet gjør det mulig å gi språkmodellen tilgang til "tools" - verktøy i form av egendefinerte funksjoner - som den kan bruke når den svarer på en forespørsel. Dette kan for eksempel brukes til å hente oppdatert informasjon fra nettet eller andre kilder. 

Språkmodellen får ikke kjøre kode eller kalle eksterne systemer på egen hånd, men får oppgitt formelle beskrivelser av tilgjengelige funksjoner ("tools") den kan bruke. Når den behandler en forespørsel vil så modellen selv bestemme seg for om den ønsker å bruke noen av de tilgjengelige "tools". Om den ønsker å bruke en funksjon, returnerer den et funksjonskall i det spesifiserte formatet. Å faktisk utføre funksjonskallet må håndteres i koden som kaller API-et.  

Oppsummert kan man oppsummere fremgangsmåten slik: 
1. Gjør kall til språkmodellen med forespørsel fra bruker, og gi modellen en liste med tilgjengelige hjelpefunksjoner i "tools" parameteren
2. Ut fra forespørselen og funksjonsbeskrivelsene, velger modellen om den vil bruke en eller flere hjelpefunksjoner. Om den vil bruke en funksjon, returnerer den et JSON-objekt som matcher funksjonsbeskrivelsen
3. Sjekk om modellen har returnert et funksjonskall (sjekk message.tool_calls-verdien) og kall i så fall den aktuelle hjelpefunksjonen med parameterverdiene oppgitt av modellen.
4. For å få et endelig svar på bruker-forespørselen, kall LLM - modellen på nytt med responsen fra (3) som et element i "messages" listen (med role="tool") 

Et enkelt (og litt klossete) eksempel på denne teknikken vises under.

In [ ]:
# definerer en dummy-funksjon språkmodellen skal kunne kalle


# Hardkodet værtjeneste - kan erstattes med eksternt API-kall eller lignende
def get_current_weather(location):
    """Get the current weather in a given location"""
    if "bergen" in location.lower():
        return json.dumps({"location": "Bergen", "temperature": "-10", "unit": "celcius", "weather": "heavy snow"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "85", "unit": "fahrenheit", "weather": "sunny"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius", "weather": "cloudy"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# json beskrivelse av værtjenesten med tekstbeskrivelse av funksjon ("description") og argumenter
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

In [ ]:
# run_conversation: hjelpefunksjon for å besvare en bruker-forespørsel, potensielt med hjelp av en "tool"

def run_conversation(query):
    # Steg 1 : initier "konversasjon" - send forespørsel og beskrivelse av tilgjengelige hjelpefunksjoner til modell-API
    messages = [{"role": "user", "content": query}]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        temperature=1.5,
        tools=tools, # funksjonsbeskrivelse 
        tool_choice="auto",  # auto er default, kan overstyres hvis man vil "nekte" modellen å gjøre valg
    )

    # Steg 2: ta vare på den initielle responsen, og sjekk om modellen ønsker å bruke hjelpefunksjon
    response_message = response.choices[0].message
    messages.append(response_message)
    tool_calls = response_message.tool_calls

    if tool_calls:
        # Steg 3 (optional): kall til hjelpefunksjon(er)
        # bare en funksjon i dette eksemplet, men man kan ha flere
        available_functions = {
            "get_current_weather": get_current_weather,
        } 
        
        for tool_call in tool_calls:
            # kall den valgte funksjonen med parameterverdiene modellen har spesifisert
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments) 
            function_response = function_to_call(
                location=function_args.get("location")
            ) 
            
            # utvid samtalen med respons fra hjelpefunksjonen
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
            
        # Steg 4 (optional): send data om hvert funksjonskall og hver respons tilbake til modellen i et nytt kall.
        # Få til slutt en endelig respons fra modellen gitt resultatet av funskjonkallene. 
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  
        messages.append(second_response.choices[0].message)  
    return messages


In [ ]:
# funksjonskall anvendt: 

# med "run_conversation" - hjelpefunksjonenkan vi se steg for steg hvordan modellen svarer på ulike forespørsler når den har tilgang på et verktøy. 
#  eksempler: vær i Bergen, band-medlemmer - Python lister...
# Terskelen for å bruke verktøy ser ut til å være ganske lav.

test_messages=run_conversation("Who were the founding members of New Order?")

for msg in test_messages: 
    pprint(dict(msg))
    print('\n')

## 4. OpenAIs embeddings - endepunkt

<center><img src="resources/image resolution.jpg" alt="image resolution" width="800" />

OpenAI har et eget endepunkt for å konvertere tekst til såkalte embeddings - som er lange flyttalls - vektorer. Man kan tenke på embeddings som en dimensjonsreduseringsteknikk for tekstdata. Embeddings brukes typisk til å sammenligne tekstelementer i forhold til innholdsmessig likhet på en rask og beregningsmessig billig måte. 

I LLM - applikasjoner brukes ofte embeddings til å bygge relevant kontekst i prompts. Ved hjelp av embeddings og forskjellige numeriske sammenligningsmål (similarity measures) kan tekst fra store mengder bakgrunnsinformasjon sorteres etter relevans for en spesifikk bruker-forespørsel.

Når man bruker OpenAIs embeddings - endepunkt, vil både maksimalt antall tokens i input-teksten og dimensjonaliteten til vektoren som returneres være avhenging av hvilken embeddings-modell som brukes. I eksemplene under bruker vi *text-embedding-3-small* - med denne kan input-teksten være inntil 8191 tokens, og vektorene som returneres er 1536-dimensjonale, dvs de inneholder alltid 1536 tall.  



### 4.1 Enkel bruk av embeddings
I koden under leser vi inn noen eksempeltekster, og bruker embeddings til å sammenligne og sortere dem ut fra semantisk innhold. 

#### 4.1.1 Innlesning av eksempeldata

In [ ]:
# leser inn et lite datasett med teksteksempler

# hjelpefunksjon for å sjekke antall tokens i tekst
def token_count(text, encoding_model):
    encoding=tiktoken.get_encoding(encoding_model)
    n_tokens = len(encoding.encode(text))
    return n_tokens

# leser tekster fra csv-fil, beregner antall tokens og inspiserer. 
df_text = pd.read_csv('data/text_samples_mat.csv', header=0, sep=';')
df_text['n_tokens']=df_text.apply(lambda row: token_count(row['quote_text'], "cl100k_base"), axis=1)
df_text

In [ ]:
# Inspeksjon av enkelttekst 
print(df_text.loc[2,'quote_text'])

#### 4.1.2 Lage embeddings ved hjelp av OpenAI
Det er relativt rett frem å lage embeddings med kall til *embeddings.create*. Her bruker vi en enkelt input-streng, men APIet aksepterer også et array av strenger som input. 

In [ ]:
# lager hjelpefunksjon for å hente embeddings fra OpenAI

# embedding hjelpefunksjon - returnerer embedding-objekt fra OpenAI laget med valgt modell
def embed_helper(AIclient, text, model_name):
    embedding=client.embeddings.create(
          model=model_name,
          input=text,
          encoding_format="float")
    return embedding

In [ ]:
# lager embedding med kall til API
test_embedding=embed_helper(client, df_text.loc[2,'quote_text'], "text-embedding-3-small")

# sjekker dimensjonalitet, inspiserer "rå" embedding output:
print('No of elements: '+str(len(test_embedding.data[0].embedding))) 
print(test_embedding.data[0].embedding)

In [ ]:
# enkel visualisering av embeddingvektor

# lager hjelpefunksjon for visualisering
def barplot_embedding(embedding, label_list):
    sns.heatmap(np.array(embedding.data[0].embedding).reshape(-1,1536), cmap="Greys", center=0, square=False, xticklabels=False, cbar=False)
    plt.gcf().set_size_inches(13,1)
    plt.yticks([0.5], labels=[label_list])
    plt.show()

# barplot for embedding av utvalgte tekster
test_embedding=embed_helper(client, "", "text-embedding-3-small")
barplot_embedding(test_embedding, "empty string")

test_embedding=embed_helper(client, df_text.loc[0,'quote_text'], "text-embedding-3-small")
barplot_embedding(test_embedding, df_text.loc[0,'title'])

test_embedding=embed_helper(client, df_text.loc[1,'quote_text'], "text-embedding-3-small")
barplot_embedding(test_embedding, df_text.loc[1,'title'])

test_embedding=embed_helper(client, df_text.loc[3,'quote_text'], "text-embedding-3-small")
barplot_embedding(test_embedding, df_text.loc[3,'title'])

#### 4.1.3 Rangering av tekstelementer etter innholdsmessig likhet
Med hjelp av et numerisk sammenligningsmål (similarity measure) - her er det flere tekniske muligheter - kan vi rangere sitatene i datasettet etter likhet med hverandre, eller etter likhet med en annen gitt tekst.

Rent teknisk fungerer dette uavhengig av språk, men sammenligning av tekst fra ulike språk gir mindre pålitelige resultater.

In [ ]:
# Hjelpefunksjoner for tekstsammenligning

# naiv hjelpefunksjon for å beregne cosinus similaritet mellom to tekststrenger
def similarity_helper(AIclient, text_1, text_2, embed_model):
    text_1_embedded_np=np.array(embed_helper(AIclient, text_1, embed_model).data[0].embedding).reshape(1,-1)
    text_2_embedded_np=np.array(embed_helper(AIclient, text_2, embed_model).data[0].embedding).reshape(1,-1)
    similarity=cosine_similarity(text_1_embedded_np, text_2_embedded_np)[0,0]
    return similarity

# hjelpefunksjon for å beregne similaritet mellom en variabel (df_text_column) i en DataFrame og en oppgitt tekst (input_text)
def df_add_similarity(AIclient, df_text, df_text_column, input_text, embed_model):   
    df_text['input_similarity']=df_text.apply(lambda row: similarity_helper(AIclient, row[df_text_column], input_text, embed_model), axis=1)
    return df_text

In [ ]:
# rangerer tekstelementer i en DataFrame ut fra similaritet med en input-tekst 

# definer input - tekst her (referanse for sammenligning):
# input_text="Be, Act, Stay Curious!"
# input_text="Norias omsetning har eksplodert de siste årene, spesielt i Bergen."
# input_text="Gourmetpølser har blitt vår nye nasjonalrett, jubler Pølsens Venner"
input_text=df_text.loc[1, 'quote_text']

# Inkluderer input_tekst i datasett, for kontroll og som referanse 
df_input=pd.DataFrame(data={'quote_id':100, 'quote_name':'Input', 'author': 'user', 'title':'Input' , 'quote_text': input_text}, index=[10])
df_total=pd.concat([df_text, df_input])

# beregner similaritet
df_add_similarity(client, df_total, 'quote_text', input_text, "text-embedding-3-small")

# lager en enkel illustrasjon av tekst-similaritet 
values=list(df_total.sort_values('input_similarity', ascending=True)['input_similarity'])
names=list(df_total.sort_values('input_similarity', ascending=True)["quote_text"].str[:50])

fig, ax = plt.subplots(figsize=(10,5))
ax.barh(names, values)
plt.show()

Prøv gjerne å laste opp egne data (lettest å gjenbruke struktur i .csv - fil), og gjør lignende tester med dem! 

#### 4.1.3 Gruppering og klassifisering av tekst

Siden embeddings jo bare er vektorer med flyttall, er de teknisk sett enkle å plugge inn i forskjellige analysemetoder. En interessant mulighet er å  bruke ulike dimensjonsreduserings-teknikker til å gruppere og klassifisere tekster. 

Under gjør vi en prinsipalkomponent - analyse av eksempeldataene vi brukte over. Litt forenklet "tvinger" vi det 1536-dimensjonale rommet vi egentlig er i ned i færre dimensjoner, som er enklere for menneskehjerner å forholde seg til. 

In [ ]:
# Lager nye hjelpefunksjoner for mer effektiv håndtering av embeddings
# funksjon som for en gitt tekst (text) returnerer embedding-vektor som numpy-array 
def embed_helper_2(AIclient, text, model_name):
    embedding=client.embeddings.create(
          model=model_name,
          input=text,
          encoding_format="float")
    return np.array(embedding.data[0].embedding).reshape(1,-1)

# hjelpefunksjon for å lagre embeddings, lables og eventuelle kategorier (cats) fra dataframe i numpy arrays.
def embed_df(AIclient, df, text_col, label_col, cat_col, model_name): 
    embedding_array=df.apply(lambda row: embed_helper_2(AIclient, row[text_col], model_name)[0], axis=1, result_type="expand")
    labels=df[label_col]
    cats=df[cat_col]
    return np.asarray(embedding_array), np.asarray(labels),  np.asarray(cats)

# similaritetsberegning for array
def similarity_rank_2(AIclient, embed_array, input_text, embed_model): 
    input_text_embedded=embed_helper_2(AIclient, input_text, embed_model)
    similarities=cosine_similarity(embed_array,input_text_embedded)
    return similarities


In [ ]:
# Lager embeddings for dataframe fra 4.1.1 (df_text)
embed_array, embed_lbl, embed_cat=embed_df(client, df_text, 'quote_text', 'title', 'author', "text-embedding-3-small")

In [ ]:
# prinsipalkomponent-analyse m plotting av 2 første komponenter

def plot_embeddings(embedding_vectors, labels, cats, show_labels=False):
    # Create a PCA model
    pca_model = PCA(random_state=42)

    # Fit and transform the data to obtain PCA coordinates
    pca_result = pca_model.fit(embedding_vectors)
    
    #print("PCA 2-components explained variance:" + str(pca_result.explained_variance_ratio_) + "\n")
    pca_trans = pca_model.fit_transform(embedding_vectors)

    # Plot the PCA results
    # explained variance
    fig=plt.figure(figsize=(8, 12))
    
    ax=fig.add_subplot(211)
    plt.title('PCA Explained variance by no. of components')
    plt.plot(np.cumsum(pca_result.explained_variance_ratio_))
    plt.grid(alpha=0.2)

     # scatterplot by artist
    ax=fig.add_subplot(212)
    plt.title('PCA Projection of Embedding Vectors')
    
    cat_list=list(set(cats))
    cat_vals=np.asarray([cat_list.index(c) for c in cats])

    cmap='tab20'
    color_map = plt.colormaps[cmap].resampled(20)
    
    for i, cat in enumerate(cat_list):
        filter_arr = []
        for catval in cats:
            if catval==cat:
                filter_arr.append(True)
            else:
                filter_arr.append(False)

        plt.scatter(pca_trans[filter_arr, 0], pca_trans[filter_arr, 1], color=color_map(i/len(cat_list)), label=cat, s=20) 
       
    if show_labels:
        for i, lbl in enumerate(labels):
            ax.text(pca_trans[:, 0][i],pca_trans[:, 1]  [i], labels[i], fontsize=8)         
    
    plt.legend()
    plt.grid(alpha=0.2)
    plt.xlabel('PCA component 1')
    plt.ylabel('PCA component 2')
    plt.show()    
    
plot_embeddings(embed_array, embed_lbl, embed_cat, show_labels=True)

### 4.2 Analyse av et større tekstdatasett

Filen "lyrics.csv" inneholder rundt 25 000 sangtekster med kjente, kjære og mindre kjære artister. Under tester vi noen av de samme teknikkene på disse dataene. 

Først leser vi inn hele datasettet og lager etpar hjelpevariable. 

In [ ]:
# innlesning av sangtekst datasett (25 000 sanger)
df_sang = pd.read_csv('data/lyrics.csv', header=0, sep=',')

# hjelpevariable for litt enklere oppslag og navigasjon
df_sang['n_tokens']=df_sang.apply(lambda row: token_count(row['lyrics'], "cl100k_base"), axis=1)
df_sang['song_id']=df_sang.index
df_sang['artist']=df_sang['artist'].apply(lambda x: x[:-7]) # fjerner unødvendig "Lyrics" i artistnavn 
df_sang['song_label']=df_sang['song_id'].apply(str) + " - " + df_sang['artist'] + " - " +df_sang['song_name']

# rask inspeksjon
df_sang

In [ ]:
# sjekk av artister - viser de 200 med flest sanger (kan justeres)
pd.options.display.max_rows=200
df_sang.value_counts('artist', ascending=False)[:200]

In [ ]:
# sjekk sanger for en valgt artist
df_sang[df_sang['artist']=='Snoop Dogg'][:200]['song_label']

For å begrense ventetid lager vi et tilfeldig sample på 100 sanger (kan justeres) som vi bruker videre. I koden under er samplet begrenset til noen utvalgte artister - gjør gjerne endringer i utvalget!

In [ ]:
# lager sample på 100 sanger, noen valgte artister
df_sang_sample=df_sang[df_sang['artist'].isin(['Bob Dylan', 'Snoop Dogg', 'Backstreet Boys', 'Lana Del Rey', 'Eminem', 'Taylor Swift'])].sample(n=100).reset_index()
print(df_sang_sample['song_label'][:100])

Vi lager så embeddings av tekstene i utvalget vårt med hjelp av funksjonene vi lagde i forrige avsnitt.

In [ ]:
# lager array med embeddings
sang_embed_array, sang_embed_lbl, sang_embed_cat=embed_df(client, df_sang_sample, 'lyrics', 'song_label', 'artist', "text-embedding-3-small")

Vi kan nå sjekke similaritet mellom tekstene, eller med annen input - tekst om vi ønsker:

In [ ]:
# beregning av similaritet med input-tekst

#ref_lyrics: sangteksten vi ønsker å sammenligne de andre med. Her kan man selvfølgelig også oppgi en annen, vilkårlig tekst 
#ref_lyrics=df_sang_sample.iloc[21]["lyrics"]
#ref_lyrics="Wholehearted, responsible, likable and competent"
#ref_lyrics="Christmas"

similarities=similarity_rank_2(client, sang_embed_array, ref_lyrics, 'text-embedding-3-small')
df_sang_sample["similarities_arr"]=similarities

In [ ]:
# viser topp n og laveste n tekster, rangert etter similaritet med input_tekst
n=5
sim_col="similarities_arr"

values_topn=list(df_sang_sample.nlargest(n,sim_col).sort_values(sim_col, ascending=True)[sim_col]) 
values_smalln=list(df_sang_sample.nsmallest(n,sim_col).sort_values(sim_col, ascending=True)[sim_col])

names_topn=list(df_sang_sample.nlargest(n,sim_col).sort_values(sim_col, ascending=True)["song_label"])
names_smalln=list(df_sang_sample.nsmallest(n,sim_col).sort_values(sim_col, ascending=True)["song_label"])

values=values_smalln+values_topn
names=[c.replace('$', 'S') for c in names_smalln+names_topn] # Paid da cost to be da bo$$ quick fix  

fig, ax = plt.subplots(figsize=(15,8))
ax.barh(names_smalln, values_smalln)
ax.barh(names_topn, values_topn)

plt.title('Top/bottom similarity with ' + ref_lyrics[:50] + '...')
plt.show()


Er det noen fornuft i rangeringen? Bruk gjerne hjelpefunksjonen under for å inspisere to tekster ved siden av hverandre. 

In [ ]:
# hjelpefunksjon for sjekk av tekster mot hverandre
from IPython.display import display, HTML

def view_lyrics(df, song_id_1, song_id_2):
    df_temp=df_sang_sample.loc[df_sang_sample['song_id'] == song_id_1].reset_index()
    label1=df_temp.loc[0, "song_label"]
    lyrics1=df_temp.loc[0, "lyrics"]

    df_temp=df_sang_sample.loc[df_sang_sample['song_id'] == song_id_2].reset_index()
    label2=df_temp.loc[0, "song_label"]
    lyrics2=df_temp.loc[0, "lyrics"]
    
    html_code = f"""
    <div style="display: flex; justify-content: space-between;">
        <div style="flex: 1; padding-right: 8px;">
            <h2>{label1}</h2>
            <pre style="font-size: 9px;"> {lyrics1} </pre>
        </div>
        <div style="flex: 1; padding-left: px;">
            <h2>{label2}</h2>
            <pre style="font-size: 9px;">{lyrics2}</pre>
        </div>
    </div>
    """
    return html_code

In [ ]:
# vis to tekster ved siden av hverandre - bruk sang_id for valg av tekster
display(HTML(view_lyrics(df_sang_sample, 23312, 10172)))

Vi kan også sjekke hvordan PCA eller t-SNE (t-distributed stochastic neighbor embedding) dimensjonsreduksjon plasserer sangene i to dimensjoner. Ser det f eks ut til at sangene fra samme artist er samlet i noen grad?

For TSNE, test gjerne effekten av å justere på perplexity - parameteren.

In [ ]:
# PCA-plott av embeddings, med artist som kategori
 
plot_embeddings(sang_embed_array, sang_embed_lbl, sang_embed_cat, show_labels=False)

In [ ]:
# t-SNE - alternativ (ikke-lineær) metode for dimensjonsreduksjon

def plot_embeddings_tsne(embedding_vectors, labels, cats, show_labels=False, perplexity=10):
    # Create a TSNE model
    tsne_model = TSNE(n_components=2, random_state=42, perplexity=perplexity)

    # Fit and transform the data to obtain PCA coordinates
    tsne_result = tsne_model.fit_transform(embedding_vectors)

    # Plot the TSNE result
    cat_list=list(set(cats))
    cat_vals=[cat_list.index(c) for c in cats]
    fig=plt.figure(figsize=(8, 6))
    ax=fig.add_subplot(111)

    plt.title('PCA Projection of Embedding Vectors')
    
    cmap='tab20'
    color_map = plt.colormaps[cmap].resampled(20)
    
    # scatterplot by artist
    for i, cat in enumerate(cat_list):
        filter_arr = []
        for catval in cats:
            if catval==cat:
                filter_arr.append(True)
            else:
                filter_arr.append(False)

        plt.scatter(tsne_result[filter_arr, 0], tsne_result[filter_arr, 1], color=color_map(i/len(cat_list)), label=cat, s=20) 
                 
    if show_labels:
        for i, lbl in enumerate(labels):
            ax.text(tsne_result[:, 0][i],tsne_result[:, 1]  [i], labels[i], fontsize=8)   
            
    plt.legend()  
    plt.title('TSNE Projection of Embedding Vectors')
    
    plt.grid(alpha=0.2)
    plt.show()    
    
# test gjerne forskjellige verdier for perplexity
plot_embeddings_tsne(sang_embed_array, sang_embed_lbl, sang_embed_cat, perplexity=10, show_labels=False)